In [36]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')

import torch
import torch.nn as nn
import torch.optim as optim

from tcn import TemporalConvNet

from utils.data import get_returns_prices

/Users/jacobfeitelberg/opt/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  after removing the cwd from sys.path.


In [37]:
train_prices_npy, test_prices_npy = get_returns_prices('data/')

Loading Files...


100%|██████████| 5/5 [00:00<00:00,  6.49it/s]


In [38]:
train_prices = torch.from_numpy(np.transpose(train_prices_npy, [0, 2, 1])).float()
test_prices = torch.from_numpy(np.transpose(test_prices_npy, [0, 2, 1])).float()

log_prices = torch.log(train_prices)
nsamples = log_prices.shape[2]
means = torch.unsqueeze(torch.mean(log_prices, dim = 2), dim=2).repeat((1,1,nsamples))
stds = torch.unsqueeze(torch.std(log_prices, dim = 2), dim=2).repeat((1,1,nsamples))
X = (log_prices - means) / stds

print(train_prices.shape, test_prices.shape)

torch.Size([247, 5, 1950]) torch.Size([247, 5, 390])


In [39]:
class TCNLinear(nn.Module):
    def __init__(self, num_inputs, num_channels, nsamples, hidden_size, kernel_size=2, dropout=0.2):
        super(TCNLinear, self).__init__()
        self.tcn = TemporalConvNet(num_inputs, num_channels, kernel_size, dropout)
        self.linear = nn.Sequential(
            nn.Flatten(),
            nn.Linear(nsamples, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_inputs),
            nn.Tanh()
        )
        
    
    def forward(self, x):
        x = self.tcn(x)
        x = self.linear(x)
        return x

In [40]:
net = TCNLinear(num_inputs = 5, num_channels = [5, 5, 5, 1], nsamples = 1950, hidden_size = 50)

net(train_prices)

tensor([[0.0650, 0.0639, 0.0571, 0.0277, 0.0933],
        [0.0650, 0.0639, 0.0571, 0.0277, 0.0933],
        [0.0650, 0.0639, 0.0571, 0.0277, 0.0933],
        ...,
        [0.0650, 0.0639, 0.0571, 0.0277, 0.0933],
        [0.0650, 0.0639, 0.0571, 0.0277, 0.0933],
        [0.0650, 0.0639, 0.0571, 0.0277, 0.0933]], grad_fn=<TanhBackward>)

In [41]:
import utils.statarb as sa

def loss_fn(hedges: torch.Tensor, prices: torch.Tensor):
    portfolio = torch.matmul(hedges, prices)
    
    return sa.adfuller(portfolio)

hedges = net(train_prices[:1])[0]
p = test_prices[0]
loss = loss_fn(hedges,p)
print(loss)

tensor(0.7285, grad_fn=<MulBackward0>)


In [42]:
# Normalize train_prices
means = torch.unsqueeze(torch.mean(train_prices, dim = 2), dim = 2)
stds = torch.unsqueeze(torch.std(train_prices, dim = 2), dim = 2)

train_prices = (train_prices - means.repeat((1,1,train_prices.shape[2]))) / stds.repeat((1,1,train_prices.shape[2]))

In [49]:
net = TCNLinear(num_inputs = 5, num_channels = [5, 5, 5, 1], nsamples = 1950, hidden_size = 100)
optimizer = optim.SGD(net.parameters(), lr = 1e-3)

epochs = 100

for epoch in range(1,epochs+1):
    optimizer.zero_grad()
    
    hedges = net(train_prices)
    
    loss = 0
    for i, hedge in enumerate(hedges):
        loss += loss_fn(hedge, test_prices[i,:,:])
        
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(epoch, loss.item())

10 150.51007080078125
20 144.20445251464844
30 156.3860321044922
40 155.15870666503906
50 154.293212890625
60 153.28128051757812


KeyboardInterrupt: 

In [47]:
hedges = net(train_prices)
num_good = 0
num_bad = 0
for i, hedge in enumerate(hedges):
    loss = loss_fn(hedge, test_prices[i,:,:]).item()
    if loss < 0.1:
        num_good += 1
    else:
        num_bad += 1
print(num_good, num_bad)
#         portfolio = torch.matmul(hedge, test_prices[i,:,:])
#         portfolio_npy = portfolio.detach().numpy()
#         plt.figure()
#         plt.plot(portfolio_npy)
#         mean = np.mean(portfolio_npy)
#         std = np.std(portfolio_npy)
#         plt.axhline(mean + std)
#         plt.axhline(mean)
#         plt.axhline(mean - std)
#         plt.title(f'ADF P Value: {loss}')
#         plt.show()

9 238
